libs


In [17]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abelc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abelc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abelc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


mock data


In [2]:
mock = pd.DataFrame()
mock['text'] = [
  "@meraaahputiiih Pindah ibu kota, semangat baru buat Indonesia yang semakin maju",
  "Jokowi minta daerah sekitar IKN jadi tulang punggung ketahanan pangan buat ibu kota baru. Ini jadi peluang besar buat daerah seperti Kalimantan Timur buat kembangkan sektor pertanian dan perikanan secara lebih terintegrasi. ðŸ˜˜ https://t.co/2tHAdGWaiz",
  "@MerMerlyea gara2 siapa ini yg memaksa bikin ibu kota baru ðŸ”¥ðŸ”¥ kalo negara ni ga mampu knpa di paksa sii ðŸ˜¶â€ðŸŒ«ï¸",
  "@TirtoID Ditawarin sosok yg cuma bikin taman aja ada analisanya,sampe mikirin mau dibikin park / garden,malah milih ngelanjutin mindah ibu kota dulu baru pusing kemudian.",
]
mock

,text
0,"@meraaahputiiih Pindah ibu kota, semangat baru..."
1,Jokowi minta daerah sekitar IKN jadi tulang pu...
2,@MerMerlyea gara2 siapa ini yg memaksa bikin i...
3,@TirtoID Ditawarin sosok yg cuma bikin taman a...


text cleaning


In [5]:
def clean_text(text):
  # remove RT tag
  text = re.sub(r'RT\s', '', text)
  # remove @_username
  text = re.sub(r"\@([\w]+)", " ", text)
  # replace emoji decode with space
  text = re.sub(r"\\u[a-zA-Z0-9]{4}", " ", text)
  # replace enter /n/ with space
  text = re.sub(r"\n\s", " ", text)
  text = re.sub(r"\n", " ", text)
  # remove non-ascii
  text = re.sub(r'[^\x00-\x7F]+',' ', text)
  # fix duplicate characters (ex: hellooooo)
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)
  # replace url
  text = re.sub(r'http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', text)
  text = re.sub(r'pic.twitter.com?.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', text)
  # convert to lowercase
  text = text.lower()
  # remove hashtag
  text = re.sub(r'\#[a-zA-Z0-9_]+','', text)
  # remove numbers
  text = re.sub(r'[0-9]+',' ', text)
  # remove symbols
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)
  # remove extra spaces to one space
  text = re.sub(r' +', ' ', text)
  # remove leading and trailing spaces
  text = re.sub(r'^[ ]|[ ]$','', text)
  # replace ikn with ibu kota negara baru
  text = text.replace("ikn", "ibukota negara baru")
  
  return text

mock["cleaned_text"] = mock["text"].apply(clean_text)
display(mock)

,text,cleaned_text
0,"@meraaahputiiih Pindah ibu kota, semangat baru...",pindah ibu kota semangat baru buat indonesia y...
1,Jokowi minta daerah sekitar IKN jadi tulang pu...,jokowi minta daerah sekitar ibukota negara bar...
2,@MerMerlyea gara2 siapa ini yg memaksa bikin i...,gara siapa ini yg memaksa bikin ibu kota baru ...
3,@TirtoID Ditawarin sosok yg cuma bikin taman a...,ditawarin sosok yg cuma bikin taman aja ada an...


pembakuan kata


In [7]:
SLANG_DICTIONARY_FILE_NAME_1 = "kamus_slang_1.csv"
SLANG_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{SLANG_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_SLANG_DICTIONARY_1 = pd.read_csv(SLANG_DICTIONARY_FILE_PATH_1)

SLANG_DICTIONARY_FILE_NAME_2 = "kamus_slang_2.csv"
SLANG_DICTIONARY_FILE_PATH_2 = f"../dictionaries/{SLANG_DICTIONARY_FILE_NAME_2}"
DATA_FRAME_SLANG_DICTIONARY_2 = pd.read_csv(SLANG_DICTIONARY_FILE_PATH_2)

SLANG_DICTIONARY_1 = pd.Series(DATA_FRAME_SLANG_DICTIONARY_1.formal.values, index=DATA_FRAME_SLANG_DICTIONARY_1.slang).to_dict()
SLANG_DICTIONARY_2 = pd.Series(DATA_FRAME_SLANG_DICTIONARY_2.formal.values, index=DATA_FRAME_SLANG_DICTIONARY_2.slang).to_dict()

display(DATA_FRAME_SLANG_DICTIONARY_1.tail())
display(DATA_FRAME_SLANG_DICTIONARY_2.tail())

,slang,formal
3258,boz,bos
3259,mayan,lumayan
3260,ribed,ribet
3261,ntapz,mantap
3262,ntaps,mantap


,slang,formal
1499,uu,peraturan
1500,keppres,keputusan presiden
1501,mancet,macet
1502,eo,panitia
1503,survei,survey


In [8]:
def slang_dict_integration_kamus_1(text):
  words = text.split()
  standardization_words = []

  for word in words:
    if word in SLANG_DICTIONARY_1:
      standardization_words.append(SLANG_DICTIONARY_1[word])
    else:
      standardization_words.append(word)

  return " ".join(standardization_words)

def slang_dict_integration_kamus_2(text):
  words = text.split()
  standardization_words = []

  for word in words:
    if word in SLANG_DICTIONARY_2:
      standardization_words.append(SLANG_DICTIONARY_2[word])
    else:
      standardization_words.append(word)

  return " ".join(standardization_words)

mock["after_slang_text"] = mock["cleaned_text"].apply(slang_dict_integration_kamus_1)
mock["after_slang_text"] = mock["after_slang_text"].apply(slang_dict_integration_kamus_2)
display(mock)

,text,cleaned_text,after_slang_text
0,"@meraaahputiiih Pindah ibu kota, semangat baru...",pindah ibu kota semangat baru buat indonesia y...,pindah ibu kota semangat baru buat indonesia y...
1,Jokowi minta daerah sekitar IKN jadi tulang pu...,jokowi minta daerah sekitar ibukota negara bar...,jokowi meminta daerah sekitar ibukota negara b...
2,@MerMerlyea gara2 siapa ini yg memaksa bikin i...,gara siapa ini yg memaksa bikin ibu kota baru ...,gara siapa ini yang memaksa bikin ibu kota bar...
3,@TirtoID Ditawarin sosok yg cuma bikin taman a...,ditawarin sosok yg cuma bikin taman aja ada an...,ditawarin sosok yang cuma bikin taman saja ada...


negation handling


In [10]:
NEGASI_DICTIONARY_FILE_NAME_1 = "negasi.csv"
NEGASI_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{NEGASI_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_NEGASI_DICTIONARY_1 = pd.read_csv(NEGASI_DICTIONARY_FILE_PATH_1)

display(DATA_FRAME_NEGASI_DICTIONARY_1.tail())

ANTONYM_DICTIONARY_FILE_NAME_1 = "antonim_bahasa_indonesia.csv"
ANTONYM_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{ANTONYM_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_ANTONYM_DICTIONARY_1 = pd.read_csv(ANTONYM_DICTIONARY_FILE_PATH_1)

ANTONYM_DICTIONARY_1 = pd.Series(DATA_FRAME_ANTONYM_DICTIONARY_1.antonim.values, index=DATA_FRAME_ANTONYM_DICTIONARY_1.word).to_dict()

display(DATA_FRAME_ANTONYM_DICTIONARY_1.tail())

,negasi
5,tanpa
6,pantang
7,jangan
8,bukanlah
9,sok


,word,antonim
1688,waspada,lalai
1689,ya,"bukan, tidak"
1690,yakin,ragu-ragu
1691,zalim,baik
1692,hanya,banyak


In [11]:
def underscore_negation(text):
  words = text.split()
  negation_words = set(DATA_FRAME_NEGASI_DICTIONARY_1["negasi"].values)
  skip_next = False
  new_words = []
    
  for i in  range(len(words)):
    if skip_next:
      skip_next = False
      continue
    if words[i] in negation_words and i < len(words) - 1:
      new_words.append(words[i] + "_" + words[i+1])
      skip_next = True
    else:
      new_words.append(words[i])

  return " ".join(new_words)

mock["underscore_negation_text"] = mock["after_slang_text"].apply(underscore_negation)
display(mock["underscore_negation_text"].tail().to_frame())

,underscore_negation_text
0,pindah ibu kota semangat baru buat indonesia y...
1,jokowi meminta daerah sekitar ibukota negara b...
2,gara siapa ini yang memaksa bikin ibu kota bar...
3,ditawarin sosok yang cuma bikin taman saja ada...


In [12]:
def swap_antonyms(text):
  words = text.split()
  antonym_dict = dict(zip(DATA_FRAME_ANTONYM_DICTIONARY_1["word"], DATA_FRAME_ANTONYM_DICTIONARY_1["antonim"]))
  new_words = []
    
  for word in words:
    if "_" in word:
      negation, next_word = word.split("_", 1)
      if next_word in antonym_dict:
        new_words.append(antonym_dict[next_word])
      else:
        new_words.append(word)
    else:
      new_words.append(word)
  
  return " ".join(new_words)

mock["swap_negation_text"] = mock["underscore_negation_text"].apply(swap_antonyms)
display(mock["swap_negation_text"].tail().to_frame())

,swap_negation_text
0,pindah ibu kota semangat baru buat indonesia y...
1,jokowi meminta daerah sekitar ibukota negara b...
2,gara siapa ini yang memaksa bikin ibu kota bar...
3,ditawarin sosok yang cuma bikin taman saja ada...


In [13]:
def replace_underscore(text):
  text = re.sub(r'_', ' ', text)

  return text

mock["final_negation_text"] = mock["swap_negation_text"].apply(replace_underscore)
display(mock["final_negation_text"].tail().to_frame())

,final_negation_text
0,pindah ibu kota semangat baru buat indonesia y...
1,jokowi meminta daerah sekitar ibukota negara b...
2,gara siapa ini yang memaksa bikin ibu kota bar...
3,ditawarin sosok yang cuma bikin taman saja ada...


stopwords removal


In [15]:
SW_DICTIONARY_FILE_NAME_1 = "kamus_stopwords_1.csv"
SW_DICTIONARY_FILE_PATH_1 = f"../dictionaries/{SW_DICTIONARY_FILE_NAME_1}"
DATA_FRAME_SW_DICTIONARY_1 = pd.read_csv(SW_DICTIONARY_FILE_PATH_1)

display(DATA_FRAME_SW_DICTIONARY_1.tail())

,stopwords
751,wong
752,yaitu
753,yakin
754,yakni
755,yang


In [16]:
custom_stopwords = stopwords.words('indonesian')
custom_stopwords.clear()
custom_stopwords.extend(DATA_FRAME_SW_DICTIONARY_1["stopwords"].values)

factory = StopWordRemoverFactory()
sastrawi_stopwords = factory.get_stop_words()

combined_stopwords = set(custom_stopwords).union(set(sastrawi_stopwords))

def drop_stopwords(text):
  return " ".join([word for word in text.split() if word not in combined_stopwords])

mock["after_stopwords_text"] = mock["final_negation_text"].apply(drop_stopwords)
display(mock["after_stopwords_text"].tail().to_frame())

,after_stopwords_text
0,pindah kota semangat indonesia maju
1,jokowi daerah ibukota negara tulang punggung k...
2,gara memaksa bikin kota negara paksa sih
3,ditawarin sosok bikin taman analisanya memikir...


stemming


In [18]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_indonesian_text(text, index):
  return " ".join([stemmer.stem(word) for word in text.split()])

# apply stemming with streaming logs
for index, row in mock.iterrows():
  mock.at[index, "after_stemming_text"] = stem_indonesian_text(row["after_stopwords_text"], index)

display(mock["after_stemming_text"].tail().to_frame())

,after_stemming_text
0,pindah kota semangat indonesia maju
1,jokowi daerah ibukota negara tulang punggung t...
2,gara paksa bikin kota negara paksa sih
3,ditawarin sosok bikin taman analisanya pikir b...


export


In [20]:
# final column
mock["preprocessed_text"] = mock["after_stemming_text"]

mock.to_csv("manual_preprocessing.csv", index=False)